In [1]:
import tensorflow as tf
from tensorflow.python.framework import ops

ops.reset_default_graph()

E:\Anaconda3_5_0_0\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### 自定义Esitimator

In [2]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [3]:
def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL,cache_dir="./data")
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL,cache_dir="./data")

    return train_path, test_path

In [4]:
import pandas as pd

In [5]:
def load_data(y_name ='Species' ):
    train_path,test_path = maybe_download()
    train = pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
    train_x,train_y = train,train.pop(y_name)
              
    test = pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)
    test_x ,test_y= test,test.pop(y_name)
    return (train_x,train_y),(test_x,test_y)

In [6]:
def train_input_fn(features,labels,batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    
    return dataset.make_one_shot_iterator().get_next()

In [7]:
def eval_input_fn(features,labels,batch_size):
    features = dict(features)
    if labels is None:
        inputs = features
    else:
        inputs = (features,labels)
    
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    dataset = dataset.batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

In [8]:
(train_x,train_y),(test_x,test_y) = load_data()

#### 定义feature columns

In [9]:
my_features_columns = []
for key in train_x.keys():
    my_features_columns.append(tf.feature_column.numeric_column(key=key))
print(my_features_columns)

[_NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


#### 定义模型

In [10]:
def model_fn(features,labels,mode,params):
    # 创建输入层 第一个参数为特征，第二参数为 特征的key
    net = tf.feature_column.input_layer(features,params['feature_columns'])
    # 创建神经网络
    for units in params['hidden_units']:
        net = tf.layers.dense(net,units=units,activation=tf.nn.relu)
    
    # 创建输出层
    logits = tf.layers.dense(inputs=net,units=params['n_classes'],activation=None)
    
    predicted_classes = tf.argmax(logits,axis=1)
    
    # 预测 In this case, the model function must return a tf.estimator.EstimatorSpec containing the prediction.
    # The predictions holds the following three key/value pairs:

    #   class_ids   holds the class id (0, 1, or 2) representing the model's prediction of the most likely species for this example.
    #   probabilities   holds the three probabilities (in this example, 0.02, 0.95, and 0.03)
    #   logit   holds the raw logit values (in this example, -1.3, 2.6, and -0.9)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions ={
            'class_ids':predicted_classes[:,tf.newaxis],
            'probabilities':tf.nn.softmax(logits),
            'logit':logits
        }
        return tf.estimator.EstimatorSpec(mode,predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(logits=logits,labels=labels)
    
    # 估计 In this case, the model function must return a tf.estimator.EstimatorSpec containing the model's loss and optionally one or more metrics.
    if mode == tf.estimator.ModeKeys.EVAL:
        accuracy = tf.metrics.accuracy(labels=labels,predictions=predicted_classes,name="eval_op")
        metric={'accuracy':accuracy}
        # 生成标量
        tf.summary.scalar('accuracy',accuracy[1])
        return tf.estimator.EstimatorSpec(mode,loss=loss,eval_metric_ops=metric)
    
    # 训练In this case, the model function must return an EstimatorSpec that contains the loss and a training operation.
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdagradOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss,global_step = tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode,loss = loss,train_op = train_op)

#### 定义Estimator

In [11]:
config = tf.estimator.RunConfig(model_dir="./model_custom")
risi_classes = tf.estimator.Estimator(model_fn=model_fn,params={'feature_columns':my_features_columns,'hidden_units':[10,10],'n_classes':3},config=config)

INFO:tensorflow:Using config: {'_model_dir': './model_custom', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002403673B128>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [12]:
risi_classes.train(input_fn=lambda:train_input_fn(train_x,train_y,100),steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./model_custom\model.ckpt.
INFO:tensorflow:loss = 2.1527865, step = 0
INFO:tensorflow:global_step/sec: 439.357
INFO:tensorflow:loss = 1.5026704, step = 100 (0.229 sec)
INFO:tensorflow:global_step/sec: 623.339
INFO:tensorflow:loss = 1.1963761, step = 200 (0.159 sec)
INFO:tensorflow:global_step/sec: 631.23
INFO:tensorflow:loss = 1.0633175, step = 300 (0.159 sec)
INFO:tensorflow:global_step/sec: 560.304
INFO:tensorflow:loss = 0.99344945, step = 400 (0.178 sec)
INFO:tensorflow:global_step/sec: 573.182
INFO:tensorflow:loss = 1.007433, step = 500 (0.174 sec)
INFO:tensorflow:global_step/sec: 611.868
INFO:tensorflow:loss = 0.96387625, step = 600 (0.163 sec)
INFO:tensorflow:global_step/sec: 623.338
INFO:tensorflow:lo

INFO:tensorflow:loss = 0.50601995, step = 8000 (0.162 sec)
INFO:tensorflow:global_step/sec: 619.465
INFO:tensorflow:loss = 0.5103793, step = 8100 (0.162 sec)
INFO:tensorflow:global_step/sec: 597.212
INFO:tensorflow:loss = 0.4781884, step = 8200 (0.167 sec)
INFO:tensorflow:global_step/sec: 590.14
INFO:tensorflow:loss = 0.51530033, step = 8300 (0.169 sec)
INFO:tensorflow:global_step/sec: 590.143
INFO:tensorflow:loss = 0.50983477, step = 8400 (0.169 sec)
INFO:tensorflow:global_step/sec: 623.338
INFO:tensorflow:loss = 0.47015923, step = 8500 (0.160 sec)
INFO:tensorflow:global_step/sec: 635.247
INFO:tensorflow:loss = 0.49226308, step = 8600 (0.157 sec)
INFO:tensorflow:global_step/sec: 635.249
INFO:tensorflow:loss = 0.48401433, step = 8700 (0.156 sec)
INFO:tensorflow:global_step/sec: 639.32
INFO:tensorflow:loss = 0.52146196, step = 8800 (0.158 sec)
INFO:tensorflow:global_step/sec: 573.185
INFO:tensorflow:loss = 0.52380204, step = 8900 (0.173 sec)
INFO:tensorflow:global_step/sec: 600.806
INFO

In [13]:
eval_result = risi_classes.evaluate(input_fn=lambda:eval_input_fn(test_x,test_y,100))
print(eval_result)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-08-09-07:06:12
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model_custom\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-08-09-07:06:12
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.7, global_step = 10000, loss = 0.55667305
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: ./model_custom\model.ckpt-10000
{'accuracy': 0.7, 'loss': 0.55667305, 'global_step': 10000}


In [14]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

In [15]:
predictions = risi_classes.predict(input_fn=lambda:eval_input_fn(features=predict_x,labels=None,batch_size=100))

for pred_dict,expec in zip(predictions,expected):
    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    
    print(template.format(SPECIES[class_id],
                              100 * probability, expec))
    

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model_custom\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Setosa" (77.2%), expected "Setosa"

Prediction is "Versicolor" (46.8%), expected "Versicolor"

Prediction is "Virginica" (59.5%), expected "Virginica"
